In [2]:
import csv
import functools as ft

In [144]:
def sq_sum(x):
  return sum((y ** 2 for y in x))

def sq_dist(x, y):
  return sq_sum((z[0] - z[1] for z in zip(x, y)))

The nearest neighbour classification rule

In [70]:
def nn_predict(x, patterns):
    '''Return the nearest neighbour class prediction of pattern p with respect to example set patterns.'''
    distances = map(lambda p: [p[0], sq_dist(x, p[1:-1]), p[-1]], patterns)
    nearest = ft.reduce(lambda a, b: min(a, b, key=lambda k: k[1]), distances)
    return nearest[-1]

Create the input RDD from a CSV text file and convert to 

In [178]:
patterns = sc.textFile('spline_boundary_enum.csv').map(lambda line: [int(line.split(',')[0])] + [float(x) for x in line.split(',')[1:-1]] + [int(line.split(',')[-1])])

Generate the reduced set of examples: Until no examples are predicted as having a class that differs from the true class with respect to the reduced set of examples, for each pattern predict its class and add it to the reduced set if the predicted class differs from the true class.  The final number of examples in the reduced set is influenced by the number of random samples in the initial reduced set.

In [192]:
reduced_set = patterns.takeSample(False, 100, 2)
while True:
    errors = patterns.filter(lambda p: (nn_predict(p[1:-1], reduced_set) != p[-1]))    
    if errors.count():
        reduced_set += errors.collect()
    else:
        break